In [7]:
import os
import json
import re

def get_text(text):
    text = re.sub('\n+', ' ', text.strip())
    text = re.sub('\r+', ' ', text.strip())
    text = ' '.join(map(lambda x: x.strip(), text.strip().split('\r\n')))
    clean_text, start = '', 0
    while text:
        text = text.split('<', 1)
        clean_text += text[0]
        if len(text) < 2:
            break
        tmp = text[1].split('>', 1)
        if len(tmp) < 2:
            break
        text = tmp[1]
    clean_text = unicode(clean_text, errors='ignore')
    clean_text = re.sub(' +', ' ', clean_text)
    return clean_text.strip()


def get_cat(cathtml):
    return cathtml.split('<a', 1)[1].split('>', 1)[1].split('</a>')[0] 


def get_sellerinfo(sellerinfo):
    pass
    

def parse_item(filename):
    with open(filename, 'r') as fp:
        content = open(filename, 'r').read()
    
    itemID = filename.split('/')[-1]
    date = filename.split('/')[-3]
    if content.find('loginpage-table') >= 0:
        return None
    content = content.split('<title>', 1)
    if len(content) > 1:
        content = content[1]
    else:
        return None
    title, content = content.split('</title>', 1)

    content = content.split('<i class="fa fa-btc"></i>', 1)
    if len(content) < 2:
        return None
    content = content[1]
    btc_price, content = content.split('USD', 1)
    btc_price = float(btc_price)

    content = content.split('<div class="topnav-element">', 1)
    if len(content) < 2:
        return None
    content = content[1]
    
    content = content.split('<div class="topnav-divider-element">')
    cat = get_cat(content[0])
    subcat = get_cat(content[1]) if len(content) > 2 else None
    subsubcat = get_cat(content[2]) if len(content) > 3 else None
    
    content = content[-1]
    content = content.split('<div id="single-product" class="nofirstmargin">')
    if len(content) < 2:
        return None
    else:
        content = content[1]
    
    content = content.split('<div class="embedded-feedback-list">')
    text = content[0]
    feedback = content[1] if len(content) > 1 else ''
    text_seller = text.split('Brought to you by:<br/><a class="gen-user-link"')
    if len(text_seller) == 1:
        text = get_text(text_seller[0])
        noseller_info = True
    else:
        text, sellerinfo = text_seller
        text = get_text(text)
        noseller_info = False

        # print content
    if noseller_info:
        pass
    elif len(sellerinfo.split('>', 1)) < 2:
        noseller_info = True
    elif len(sellerinfo.split('>', 1)[1].split('<', 1)) < 2:
        noseller_info = True
    else:
        noseller_info = False
        seller, sellerinfo = sellerinfo.split('>', 1)[1].split('<', 1)
        sellerinfo = sellerinfo.split('<i class="fa fa-bar-chart-o">', 1)
        if len(sellerinfo) > 1:
            sellerinfo = sellerinfo[1].split('</i>', 1)[1]
            sellerinfo = sellerinfo.split('</span>',1)
            if len(sellerinfo) > 1:
                ratinginfo, sellerinfo = sellerinfo
                ratinginfo = ratinginfo.split('&nbsp;', 1)[-1].split(',')
                if len(ratinginfo) > 1:
                    sellerrating, dealmade = ratinginfo
                    sellerrating = float(sellerrating.split('/', 1)[0].strip('~'))
                    dealmade = dealmade.strip().strip(' deals')
                else:
                    dealmade =ratinginfo[-1].strip().strip(' deals')
                    sellerrating = None
            else:
                sellerinfo = sellerinfo[-1]
                sellerrating = None
                dealmade = None
        else:
            sellerinfo = sellerinfo[-1]
            sellerrating = None
            dealmade = None
        sellerinfo = sellerinfo.split('From:', 1)
        if len(sellerinfo) < 2:
            sellerlocation = None
            sellerinfo = sellerinfo[-1]
        elif len(sellerinfo[1].split('>', 1)) < 2:
            sellerlocation = None
            sellerinfo = sellerinfo[-1]
        elif len(sellerinfo[1].split('>', 1)[1].split('<', 1)) < 2:
            sellerlocation = None
            sellerinfo = sellerinfo[-1]
        else:
            sellerlocation, sellerinfo = sellerinfo[1].split('>', 1)[1].split('<', 1)
            sellerlocation = sellerlocation.strip()
        sellerinfo = sellerinfo.split('To: ', 1)
        if len(sellerinfo) < 2:
            sellto = None
            sellerinfo = sellerinfo[-1]
        elif len(sellerinfo[1].split('<', 1)) < 2:
            sellto = None
            sellerinfo = sellerinfo[-1]
        else:
            sellto, sellerinfo = sellerinfo[1].split('<', 1)
            sellto = sellto.strip()

        price_in_btc_tmp = sellerinfo.split('<div class="product-page-price">')
        if len(price_in_btc_tmp) < 2:
            price_in_btc = float(content[0].split('<div style="text-align: left;">', 1)[1].split(' BTC</div>', 1)[0])
        else:
            price_in_btc = float(price_in_btc_tmp[1].split('<', 1)[0].strip('BTC'))
        price_in_usd = price_in_btc * btc_price
        feedbacknum = feedback.count('<span class="gen-user-link">')
    
    if noseller_info:
        seller, sellerrating, dealmade = None, None, None
        sellerlocation, sellto = None, None
        price_in_btc, price_in_usd, feedbacknum = None, None, None

    iteminfo = {}
    iteminfo['title'] = title
    iteminfo['itemID'] = itemID
    iteminfo['btc_price'] = btc_price
    iteminfo['cat'] = cat
    iteminfo['subcat'] = subcat
    iteminfo['subsubcat'] = subsubcat
    iteminfo['description'] = text
    iteminfo['seller'] = seller
    iteminfo['seller_rating'] = sellerrating
    iteminfo['seller_dealmade'] = dealmade
    iteminfo['seller_location'] = sellerlocation
    iteminfo['sell_to'] = sellto
    iteminfo['price_in_btc'] = price_in_btc
    iteminfo['price_in_usd'] = price_in_usd
    iteminfo['feedbacks_count'] = feedbacknum
    iteminfo['date'] = date

    return iteminfo

In [8]:
root_path = '/media/intel/m2/agora'
item = {}

for path in os.listdir(root_path):
    date_path = os.path.join(root_path, path)
    if os.path.isdir(date_path):
        webpage_f = os.path.join(date_path, 'p')
        if os.path.isdir(webpage_f):
            filenames = [os.path.join(webpage_f, wp) for wp in os.listdir(webpage_f)]
            for filename in filenames:
                try:
                    item[filename] = parse_item(filename)
                except:
                    print filename

with open('agora.json', 'w') as fp:
    json.dump(item, fp)

In [78]:
parse_item('/media/intel/m2/agora/2015-04-20/p/bDsqzE028v')

{'btc_price': 223.33,
 'cat': 'Services',
 'date': '2015-04-20',
 'description': u'UNIQUE TUTORIAL - Amazon carding tutorial 0.02238839 BTC Amazon carding tutorialOver The Last Few Years Amazon Has Been Getting Harder &amp; Harder To CardThey Review Each Account When They Place There 1ST Order Therefore Add Something To Your Cart.This is how you beat them guys.This ebook is unique on the market !',
 'feedbacks_count': 0,
 'itemID': 'bDsqzE028v',
 'price_in_btc': 0,
 'price_in_usd': 0,
 'sell_to': '',
 'seller': '',
 'seller_dealmade': 0,
 'seller_location': '',
 'seller_rating': 0,
 'subcat': 'Hacking',
 'subsubcat': '',
 'title': 'UNIQUE TUTORIAL - Amazon carding tutorial'}

In [1]:
import pandas as pd
import re
import os
import json
import time
from shutil import copy2
import shutil
from PIL import Image
import hashlib

In [2]:
data = pd.read_json('agora.json', orient='index')

len(data)

2468424

In [3]:
data  = data[data.seller.notnull()]

len(data)

2385965

In [4]:
len(data.seller.unique())

3162

In [5]:
root_dir = '/media/intel/m2/imgs/Agora/'
try:
    os.makedirs(root_dir)
except:
    pass

In [6]:
f = lambda x: os.path.join(root_dir, x)

In [7]:
data['seller_path'] = data.seller.apply(f)

In [8]:
data.head().seller_path

/media/intel/m2/agora/2014-01-01/p/ABnFuNDr6M       /media/intel/m2/imgs/Agora/passman
/media/intel/m2/agora/2014-01-01/p/AGs4ikK7bm    /media/intel/m2/imgs/Agora/FakeMarket
/media/intel/m2/agora/2014-01-01/p/ALDcahdCtn       /media/intel/m2/imgs/Agora/drzheng
/media/intel/m2/agora/2014-01-01/p/ANMSzpx4Kp    /media/intel/m2/imgs/Agora/FakeMarket
/media/intel/m2/agora/2014-01-01/p/AV1FYoj5md        /media/intel/m2/imgs/Agora/RepAAA
Name: seller_path, dtype: object

In [10]:
valid_item_img = 0
for index, row in data.iterrows():
    img_path = index.replace('/p/', '/liabilities/p/') + '.jpg'
    imgName = os.path.join(row.seller_path, img_path.split('/')[-1])
    if os.path.isfile(imgName):
        continue
    if not os.path.isfile(img_path):
        continue
    try:
        img = Image.open(img_path)
        img.load()
        img.close()
    except IOError:
        img.close()
        continue        
    if not os.path.isdir(row.seller_path):
        os.makedirs(row.seller_path)
    copy2(img_path, imgName)
    valid_item_img += 1
print valid_item_img

76399


In [ ]:
data = data.drop_duplicates(['seller', 'title', 'itemID'])

In [2]:
import pandas as pd
import re
import os
import json
import time
from shutil import copy2
import shutil
from PIL import Image
import hashlib

In [3]:
data = pd.read_json('agora.json', orient='index')

len(data)

2468424

In [4]:
data = data[data.cat.notnull()]

In [5]:
data.cat.unique()

array([u'Information', u'Forgeries', u'Drugs', u'Counterfeits',
       u'Drug paraphernalia', u'Services', u'Data', u'Weapons', u'Jewelry',
       u'Listings', u'Tobacco', u'Electronics', u'Other', u'Info/eBooks',
       u'Chemicals'], dtype=object)

In [6]:
root_dir = '/media/intel/m2/catAgora/'
try:
    os.mkdir(root_dir)
except:
    pass

In [7]:
f = lambda x: os.path.join(root_dir, re.sub('[\W_]+', '00', x.lower()))

In [8]:
data['cat_path'] = data.cat.apply(f)

In [9]:
valid_item_img = set()
for index, row in data.iterrows():
    if (row['cat'], row.itemID) not in valid_item_img:
        img_path = index.replace('/p/', '/liabilities/p/') + '.jpg'
        if not os.path.isfile(img_path):
            continue
        try:
            img = Image.open(img_path)
            img.load()
            img.close()
        except IOError:
            img.close()
            continue        
        if not os.path.isdir(row.cat_path):
            os.makedirs(row.cat_path)
        copy2(img_path, row.cat_path)
        valid_item_img.add((row['cat'], row.itemID))

In [3]:
import os

In [1]:
import hashlib
def md5sum(filename, blocksize=65536):
    hash = hashlib.md5()
    with open(filename, "rb") as f:
        for block in iter(lambda: f.read(blocksize), b""):
            hash.update(block)
    return hash.hexdigest()

In [4]:
root_dir = '/media/intel/m2/imgAgoradedup'
seller_name = [os.path.join(root_dir, x) for x in os.listdir(root_dir)]
for seller_folder in set(seller_name):
    filenames = map(lambda x: os.path.join(seller_folder, x), os.listdir(seller_folder))
    md5value = {}
    for filename in filenames:
        md5curr = md5sum(filename)
        if md5curr in md5value:
            os.remove(filename)
        else:
            md5value[md5curr] = filename

In [1]:
import os
import hashlib
def md5sum(filename, blocksize=65536):
    hash = hashlib.md5()
    with open(filename, "rb") as f:
        for block in iter(lambda: f.read(blocksize), b""):
            hash.update(block)
    return hash.hexdigest()
root_dir = '/media/intel/m2/catAgora'
seller_name = [os.path.join(root_dir, x) for x in os.listdir(root_dir)]
for seller_folder in set(seller_name):
    filenames = map(lambda x: os.path.join(seller_folder, x), os.listdir(seller_folder))
    md5value = {}
    for filename in filenames:
        md5curr = md5sum(filename)
        if md5curr in md5value:
            os.remove(filename)
        else:
            md5value[md5curr] = filename

In [1]:
import pandas as pd
import re
import os
import json
import time
from shutil import copy2
import shutil
from PIL import Image
import hashlib

In [2]:
data = pd.read_json('agora.json', orient='index')

len(data)

2468424

In [3]:
data2 = data[data.cat=='Drugs']

In [4]:
data2 = data2[data2.subcat.notnull()]

In [5]:
data2.subcat.value_counts()

Cannabis           416551
Ecstasy            216896
Prescription       193159
Stimulants         189590
Psychedelics       152206
Benzos             133004
Opioids            110021
Steroids           101002
RCs                 72088
Dissociatives       29359
Other               22578
Weight loss         12063
Barbiturates         1706
Disassociatives       338
Name: subcat, dtype: int64

In [6]:
root_dir = '/media/intel/m2/subcatAgora/'
try:
    os.mkdir(root_dir)
except:
    pass

In [7]:
f = lambda x: os.path.join(root_dir, re.sub('[\W_]+', '00', x.lower()))

In [8]:
data2['subcat_path'] = data2.subcat.apply(f)

In [9]:
valid_item_img = set()
for index, row in data2.iterrows():
    if (row['subcat'], row.itemID) not in valid_item_img:
        img_path = index.replace('/p/', '/liabilities/p/') + '.jpg'
        if not os.path.isfile(img_path):
            continue
        try:
            img = Image.open(img_path)
            img.load()
            img.close()
        except IOError:
            img.close()
            continue        
        if not os.path.isdir(row.subcat_path):
            os.makedirs(row.subcat_path)
        copy2(img_path, row.subcat_path)
        valid_item_img.add((row['subcat'], row.itemID))

In [10]:
import os
import hashlib
def md5sum(filename, blocksize=65536):
    hash = hashlib.md5()
    with open(filename, "rb") as f:
        for block in iter(lambda: f.read(blocksize), b""):
            hash.update(block)
    return hash.hexdigest()
root_dir = '/media/intel/m2/subcatAgora'
seller_name = [os.path.join(root_dir, x) for x in os.listdir(root_dir)]
for seller_folder in set(seller_name):
    filenames = map(lambda x: os.path.join(seller_folder, x), os.listdir(seller_folder))
    md5value = {}
    for filename in filenames:
        md5curr = md5sum(filename)
        if md5curr in md5value:
            os.remove(filename)
        else:
            md5value[md5curr] = filename